<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [4]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_features.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip



Archive:  train_features.csv.zip
replace train_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Archive:  train_labels.csv.zip
replace train_labels.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
replace test_features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.feature_selection as fe
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.feature_selection import RFECV

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

from mlxtend.plotting import plot_decision_regions
#import category_encoders as ce


In [7]:
# DO NOT USE
# date as index
df_train = pd.read_csv('train_features.csv',index_col='date_recorded', 
              parse_dates=True, infer_datetime_format=True)
df_test = pd.read_csv('test_features.csv', index_col='date_recorded',
                     parse_dates=True, infer_datetime_format=True)
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 39), (14358, 39), (59400, 2))

In [31]:
# id as index
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape, df_sample.shape


((59400, 39), (14358, 39), (59400, 2), (14358, 2))

In [32]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
        df_train, df_label, test_size=0.25, random_state=42, shuffle=True)

X_train_val.shape, X_test.shape, y_train_val.shape, y_test.shape

((44550, 39), (14850, 39), (44550, 2), (14850, 2))

In [0]:
# majority baseline
# balanced data - logistic class_weight = 
#            (None, accuracy = 0.55),
#            (balanced,accuracy=0.39-0.4)


df_label['status_group'].value_counts(normalize=True)

In [0]:
pd.set_option('display.max_columns', None)

#df_str=X_train_val.select_dtypes(exclude=['int','float']).copy()
df_str=X_train_val.select_dtypes(object).copy()
df_str.head(3)


In [0]:
# As observations increase, more features will fail to pass the threshold
# At 50,000+ observation, passing threshold drops to 0.1

#remove all features that are either one or zero (on or off) in more than 80% of the samples.
#threshold = 0.8 * (1- .8)

import sklearn.feature_selection as fe

def variance_threshold_selector(data, threshold=0.01):
    selector = fe.VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [0]:
def wrangle(X):
  X_dummies = pd.get_dummies(X.funder)
  dummy_filter=variance_threshold_selector(X_dummies)
  
  X_num = X._get_numeric_data()
  
  #y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
  #y = y.values
  
  
  return X_num.join(dummy_filter, how='outer')
  

In [0]:
# NOT USE - for understanding variance_threshold only
# same as variance threshold function
def noise_filter(df, col, threshold=0.01):
  X_dummies=pd.DataFrame()
  X_dummies[col]=df[col].value_counts(normalize=True)

  for noise_cat in X_dummies.loc[(X_dummies[col] < threshold)].index.values:
    df[col].replace(noise_cat, 'noise', inplace=True)

  #df_str['funder'].replace('0', 'noise', inplace=True)
  #df_str['funder'].replace(np.nan,'noise' ,inplace=True)

  return pd.get_dummies(df[col])

dummy_filter = noise_filter(df_str, 'funder' ,0.01)
dummy_filter=dummy_filter.drop(['0','noise'], axis=1)
dummy_filter.shape

In [38]:
# Baseline metric - numeric only - test_score=0.55
X = wrangle(X_train_val)
y = y_train_val.status_group.values


scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None), X, y, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

,estimator,fit_time,score_time,test_score,train_score
0,"LogisticRegression(C=1.0, class_weight=None, d...",1.753693,0.023899,0.554979,0.551702
1,"LogisticRegression(C=1.0, class_weight=None, d...",1.928169,0.024064,0.554747,0.560438
2,"LogisticRegression(C=1.0, class_weight=None, d...",1.916252,0.023953,0.550744,0.552035


In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

In [19]:
X_test.shape, df_test.shape

((14850, 39), (14358, 39))

In [0]:
y_test.shape, df

In [0]:
# Define an estimator and param_grid
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif,mutual_info_classif

#X,y = wrangle(X_train_val, y_train_val)
X = wrangle(X_test)
y = y_test.status_group.values


pipe = make_pipeline(
    RobustScaler(), 
    #fe.SelectKBest(mutual_info_classif), #slow 0.58
    fe.SelectKBest(f_classif),
    LogisticRegression(solver='lbfgs'))

param_grid = {
    'selectkbest__k': ['all'],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C': [.01, .1, 1.0, 10.0]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', return_train_score=True,
                  verbose=1)



In [0]:
gs.best_estimator_.predict(X)

# Need help here. gs.predict(X) can't predict without y target.

In [46]:

X=wrangle(df_test)
gs.best_estimator_.predict(X)
#gs.predict(X)

ValueError: ignored

In [40]:
gs.fit(X, y)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

results = pd.DataFrame(gs.cv_results_)
print('Best result from grid search of {} parameter combinations'.format(len(results)))
results.sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   17.0s finished


()
('Cross-Validation Score:', 0.5947474747474748)
()
('Best estimator:', Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k='all', score_func=<function f_classif at 0x7f7bcd442d70>)), ('logisticregression', LogisticRegression(C=10.0, class_weight=None, dual=False, fit...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]))
()
Best result from grid search of 8 parameter combinations


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_logisticregression__C,param_logisticregression__class_weight,param_selectkbest__k,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
6,0.688315,0.009434,0.594747,0.596465,10,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",1,0.594627,0.597232,0.584242,0.599394,0.605375,0.592768,0.012967,0.000070,0.008627,0.002759
4,0.690188,0.009409,0.591717,0.594916,1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",2,0.593819,0.596222,0.582828,0.598687,0.598505,0.589839,0.009979,0.000044,0.006570,0.003728
2,0.703399,0.010705,0.591178,0.593536,0.1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",3,0.589982,0.596525,0.584444,0.594343,0.599111,0.589738,0.016644,0.001707,0.006047,0.002829


In [0]:
#Which features were selected?


selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X.columns.values
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)


In [0]:
from sklearn.ensemble import RandomForestRegressor as RFR

X,y=wrangle(X_train_val, y_train_val)
names = X.columns

y = pd.DataFrame(y).replace(['non functional','functional','functional needs repair'],[1,2,3])
y = y.values

# Fitting a random forest regression
rfr = RFR().fit(X, y)

# Creating scores from feature_importances_ ranking (some randomness here)
rfr_scores = pd.Series(data=rfr.feature_importances_, name='RFR', index=names)

rfr_scores

In [0]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso

X,y= wrangle(X_train_val, y_train_val)


#y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
#y = y.values

X_scaled = RobustScaler().fit_transform(X)

# Lasso can't handle multi-class label.
#rfe = RFECV(Lasso(), scoring='accuracy', cv=3)
rfe = RFECV(LogisticRegression(), scoring='accuracy', cv=3)
X_subset = rfe.fit_transform(X_scaled, y)

all_names = X.columns
selected_mask = rfe.support_
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)


In [29]:
X,y=wrangle(df_test, y_test)
X.shape, y.shape

((14358, 26), (14850,))

In [0]:
# Predict with X_test features
X=wrangle(df_test)
y_pred = gs.predict(X)

# Compare predictions to y_test labels
#test_score = accuracy_score(y_test.status_group, y_pred)
#print('Test Score:', test_score)

In [15]:
#submission as format as df_sample
y_submission = y_test.copy()
y_submission['status_group']=y_pred
y_submission.shape

(14850, 2)

In [0]:
from google.colab import files

y_submission.to_csv('data.csv', index=False)
files.download("data.csv")